----
# Data Cleaning Part-(2)
----

In this second part of my data cleaning process I emphasised my full energy on the Salary column. I was interested in every salary in the dataset. As the data was scraped, the salary was messy. I wanted a salaries as a single figure detailing expected annual compensation.

In [1]:
# Libraries
import pandas as pd
import numpy as np
import re
import string

import warnings
warnings.filterwarnings('ignore')

#### Load the dataset

In [2]:
jobs = pd.read_csv('job_postings2.csv')

#### Removing Duplicates if any:

In [3]:
jobs.drop_duplicates(subset=['job_area', 'job_classification', 'job_company','job_description',
                             'job_location', 'job_salary', 'job_title', 'job_name',
                             'job_type'], keep='first', inplace= True)

In [4]:
jobs.shape

(879, 10)

In [5]:
jobs.head(2)

,Unnamed: 0,job_area,job_classification,job_company,job_description,job_location,job_salary,job_title,job_type,job_name
0,0,CBD and Inner Suburbs,Science and Technology,SEEK Limited,The Opportunity We're looking for an experienc...,Melbourne,Base + Super + Profit + Awesome Problems to So...,Senior Data Scientist,Full Time,senior data scientist
1,1,Parramatta and Western Suburbs,Information and Communication Technology,ecareer employment services,Put Data to best useDesign & Develop BI Soluti...,Sydney,$130-140K Package,Senior Data Engineer Business Intelligence D...,Full Time,senior data scientist


#### Removing Unneccesary columns

In [6]:
jobs= jobs.drop(['Unnamed: 0'],axis=1)

#### Reseting the index 

In [7]:
jobs.reset_index(inplace=True,drop=True)

#### Fixing the salary Column

The function **clean_sal** removes the **"k"** following a digit [0-9] and replaces with '000'

In [8]:
def clean_sal(text):
    text = text.lower()
    text = re.sub(r"([0-9])[k]",r"\1 000",text)
#     text = re.sub(r",","",text)
    text = re.sub(r"\s","",text)
    
    return text

In [9]:
jobs['salary1']= jobs['job_salary'].apply(lambda x: clean_sal(x))

In [10]:
def salary(text):
    text = text.lower()
    pattern = re.compile('.*(\$[\d,])+\s?-?\$?[\d,]+')
    matches=re.match(pattern,text)
    if matches:
        return matches.group(0)
    else:
         None

In [11]:
 jobs['salary']= jobs['salary1'].apply(lambda x: salary(x))

In [12]:
for index,value in jobs['salary'].iteritems():
    if value == None:
        jobs.loc[index,'salary']=0
    else:
        jobs.loc[index,'salary']=value

#### replacing all the special Characters and the few text

In [13]:
jobs["salary"] = jobs["salary"].str.replace('$','')
jobs["salary"] = jobs["salary"].str.replace('+','')
jobs["salary"] = jobs["salary"].str.replace('|','')
jobs["salary"] = jobs["salary"].str.replace('(','')
jobs["salary"] = jobs["salary"].str.replace(',','')
jobs["salary"] = jobs["salary"].str.replace('au',' ')
jobs["salary"] = jobs["salary"].str.replace('to','-')
jobs["salary"] = jobs["salary"].str.replace('excellentsalarypackage','')
jobs["salary"] = jobs["salary"].str.replace('circa',' ')
jobs["salary"] = jobs["salary"].str.replace('up',' ')
jobs["salary"] = jobs["salary"].str.replace('dailyrate','')
jobs["salary"] = jobs["salary"].str.replace('upto',' ')
jobs["salary"] = jobs["salary"].str.replace('c','')
jobs["salary"] = jobs["salary"].str.replace('ov','')
jobs["salary"] = jobs["salary"].str.replace('basesalary','')
jobs["salary"] = jobs["salary"].str.replace('toau',' ')
jobs["salary"] = jobs["salary"].str.replace('aso6','')
jobs["salary"] = jobs["salary"].str.replace('level','')
jobs["salary"] = jobs["salary"].str.replace('er','')
jobs["salary"] = jobs["salary"].str.replace('3monthontratext-','')
jobs["salary"] = jobs["salary"].str.replace('3monthontrat+ext-','')
jobs["salary"] = jobs["salary"].str.replace('remunerationpakage','')
jobs["salary"] = jobs["salary"].str.replace('remunationpakage','')
jobs["salary"] = jobs["salary"].str.replace('aggressiveomms|over','')
jobs["salary"] = jobs["salary"].str.replace('targetis130000pluss open-senior','')
jobs["salary"] = jobs["salary"].str.replace('on-rpm:','')
jobs["salary"] = jobs["salary"].str.replace('targetis130000pluss ..opentosenior','')

This take the salary from the cleaned salary column and returns the mean if the salary is a range otherwise return the exact amount

In [14]:
y=list()
for index,val in jobs.salary.iteritems():
    y=list()
#     print(index,val)
    ind = isinstance(val,float)
    
    if  ind :
        continue
    ind2 = isinstance(val,int)
    if ind2:
        continue
    else:
        val1= val.split("-")
        val1=list(filter(None,val1))
    for num in val1:
        try:
            num = int(num)
            y.append(num)
        except:
            continue
    
    ynum=round(np.mean(y),2)
    jobs.loc[index,"sal"]=ynum
       

#### Extracting the different "_hourly_" patterns from the salary column so as to normalize the salary column

In [15]:
jobs.loc[jobs['job_salary'].str.contains('hour'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('hourly'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('phr'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('hr'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('perhour'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('p.h.'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('p/hr'),"period1"]="hour"
jobs.loc[jobs['job_salary'].str.contains('ph'),"period1"]="hour"

#### Extracting the different  "_daily_"  patterns from the salary column so as to normalize the salary column

In [16]:
jobs.loc[jobs['job_salary'].str.contains('p.d.'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('pd'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('day'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('daily'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('dailyrate'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('p/d'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('per day'),"period2"]="day"
jobs.loc[jobs['job_salary'].str.contains('perday'),"period2"]="day"

In [17]:
#### Parsing through the daily and hourly salaries and converting them to annual package.

I had 56 daily rates and 14 hourly rates, to make the salary column consistent, I multipiled daily rates by 260 and hourly by 2080.

In [18]:
jobs['final_salary']=jobs['sal']

In [19]:
for index,val1 in jobs['period1'].iteritems():
    if val1 == "hour":
        jobs['final_salary'][index] = jobs['sal'][index] * 2080
    else:
        jobs['final_salary'][index]=jobs['final_salary'][index]

In [20]:
for index,val1 in jobs['period2'].iteritems():
    if val1 == "day":
        jobs['final_salary'][index] = jobs['sal'][index] * 260
    else:
        jobs['final_salary'][index]=jobs['final_salary'][index]

#### Fixing few salaries manually

In [21]:
jobs.loc[1, 'final_salary'] = 135000.0
jobs.loc[77, 'final_salary'] = 165000.0
jobs.loc[626, 'final_salary'] = 140000.0
jobs.loc[720, 'final_salary'] = 100000.0
jobs.loc[829, 'final_salary'] = 150000.0
jobs.loc[839, 'final_salary'] = 150000.0
jobs.loc[227, 'final_salary'] = 240000.0
jobs.loc[232, 'final_salary'] = 110500.0
jobs.loc[249, 'final_salary'] = 60000.0
jobs.loc[299, 'final_salary'] = 72800.0
jobs.loc[418, 'final_salary'] = 140000.0
jobs.loc[492, 'final_salary'] = 140000.0
jobs.loc[517, 'final_salary'] = 137500.0
jobs.loc[633, 'final_salary'] = 169000.0
jobs.loc[642, 'final_salary'] = 125000.0
jobs.loc[811, 'final_salary'] = 190000.0
jobs.loc[820, 'final_salary'] = 39000.0
jobs.loc[841, 'final_salary'] = 140000.0
jobs.loc[843, 'final_salary'] = 89012.0
jobs.loc[849, 'final_salary'] = 89012.0
jobs.loc[866, 'final_salary'] = 180000.0
jobs.loc[868, 'final_salary'] = 105000.0
jobs.loc[810, 'final_salary'] = 105000.0
jobs.loc[570, 'final_salary'] = 130000.0
jobs.loc[229, 'final_salary'] = 113553.0
jobs.loc[228, 'final_salary'] = 113553.0
jobs.loc[790, 'final_salary'] = 100000.0
jobs.loc[134, 'final_salary']= 166000.0


Now that we've made feature columns, we have some remaining columns that won't be useful for our modelling.

**Dropping the unneccessary columns**

In [23]:
jobs.drop(["salary1","salary","sal","period1","period2","job_title","job_salary"],axis=1, inplace= True)

**Renaming the columns in order to have same column names throughout my project.**

In [24]:
jobs.rename(columns={"job_name":"job_title", "final_salary": "job_salary"}, inplace=True)

**Job work type**

In [25]:
jobs['job_type'].value_counts()

Full Time          595
Contract/Temp      282
Part Time            1
Casual/Vacation      1
Name: job_type, dtype: int64

In [26]:
jobs['job_type'] = jobs['job_type'].str.replace('Casual/Vacation','Contract/Temp')
jobs['job_type'] = jobs['job_type'].str.replace('Part Time','Contract/Temp')

In [27]:
jobs.head(2)

,job_area,job_classification,job_company,job_description,job_location,job_type,job_title,job_salary
0,CBD and Inner Suburbs,Science and Technology,SEEK Limited,The Opportunity We're looking for an experienc...,Melbourne,Full Time,senior data scientist,NaN
1,Parramatta and Western Suburbs,Information and Communication Technology,ecareer employment services,Put Data to best useDesign & Develop BI Soluti...,Sydney,Full Time,senior data scientist,135000.0


#### I binned all the jobs and identified 8 unique job title:

In [28]:
jobs['job_title'].value_counts()

data analyst                    289
data engineer                   221
data scientist                  165
business intelligence           113
senior business intelligence     36
senior data scientist            33
junior data engineer              7
junior data analyst               6
senior research fellow            4
junior business intelligence      3
junior data scientist             2
Name: job_title, dtype: int64

In [29]:
jobs["job_title"]=jobs["job_title"].str.replace('junior data scientist','data scientist')
jobs["job_title"]=jobs["job_title"].str.replace('junior business intelligence','business intelligence')
jobs["job_title"]=jobs["job_title"].str.replace('senior research fellow','data analyst')

In [30]:
pd.DataFrame(jobs.groupby(['job_title','job_type'])['job_salary'].median())

job_salary
job_title                    job_type                 
business intelligence        Contract/Temp    191100.0
                             Full Time        110000.0
data analyst                 Contract/Temp    123500.0
                             Full Time        113553.0
data engineer                Contract/Temp    221000.0
                             Full Time        124999.5
data scientist               Contract/Temp    135000.0
                             Full Time        140000.0
junior data analyst          Contract/Temp     70200.0
                             Full Time         52500.0
junior data engineer         Contract/Temp         NaN
                             Full Time         90000.0
senior business intelligence Contract/Temp         NaN
                             Full Time        135000.0
senior data scientist        Contract/Temp    260000.0
                             Full Time        150000.0

In [32]:
pd.DataFrame(jobs.groupby(['job_title']).count())

,job_area,job_classification,job_company,job_description,job_location,job_type,job_salary
job_title,,,,,,,
business intelligence,116,116,116,116,116,116,37
data analyst,293,293,293,293,293,293,72
data engineer,221,221,221,221,221,221,48
data scientist,167,167,167,167,167,167,42
junior data analyst,6,6,6,6,6,6,3
junior data engineer,7,7,7,7,7,7,1
senior business intelligence,36,36,36,36,36,36,10
senior data scientist,33,33,33,33,33,33,18



### Imputing the values by median values


>**Business Intelligence with Full Time Role**

In [37]:
jobs[(jobs.job_title == 'business intelligence') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index

Int64Index([529, 544, 554, 566, 594, 598, 620, 678, 679, 683, 684, 687, 688,
            689, 691, 692, 693, 694, 697, 698, 702, 704, 710, 711, 715, 716,
            717, 719, 722, 723, 730, 738, 741, 744, 748, 749, 750, 764, 769,
            770, 771, 776, 779, 780, 781, 782, 786, 787, 798, 803, 813, 819,
            853, 863, 871],
           dtype='int64')

In [38]:
index=[529, 544, 554, 566, 594, 598, 620, 678, 679, 683, 684, 687, 688,689, 691, 692, 693, 694,
       697, 698, 702, 704, 710, 711, 715, 716,717, 719, 722, 723, 730, 738, 741, 744, 748, 749, 
       750, 764, 769,770, 771, 776, 779, 780, 781, 782, 786, 787, 798, 803, 813, 819,853, 863, 871]
for val in index:
    jobs.loc[val,"job_salary"] = 110000.0

>**Business Intelligence with Contract/Temp Role**

In [39]:
jobs[(jobs.job_title == 'business intelligence') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index

Int64Index([ 20, 253, 310, 680, 681, 682, 696, 699, 700, 705, 709, 718, 726,
            754, 755, 756, 761, 775, 791, 815, 816, 838, 844, 848],
           dtype='int64')

In [40]:
index=[20, 253, 310, 680, 681, 682, 696, 699, 700, 705, 709, 718, 726,754, 755, 756, 
       761, 775, 791, 815, 816, 838, 844, 848]
for val in index:
    jobs.loc[val,"job_salary"] = 191100.0

>**Data Analyst with Full Time Role**

In [41]:
jobs[(jobs.job_title == 'data analyst') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index[0:150]

Int64Index([], dtype='int64')

In [42]:
index=[17,23,  25,  26, 145, 149, 154, 155, 156, 158, 171, 173, 174, 177,
       181, 185, 189, 194, 197, 202, 203, 205, 206, 212, 213, 233, 235,
       237, 238, 244, 245, 248, 254, 258, 261, 262, 263, 264, 274, 276,
       278, 280, 282, 283, 284, 285, 286, 288, 290, 292,300, 304, 306, 309,
       311, 313, 316, 317, 318, 320, 322, 326, 331,334, 336, 341, 343, 344,
       345, 346, 352, 356, 357, 360, 361, 363,365, 509, 530, 531, 540, 553, 557,
       558, 561, 564, 569, 573, 591,611, 612, 617, 630, 650, 651, 653, 662, 665,
       666, 677,759, 760, 763, 767, 799,800, 807, 814, 817, 818, 830, 845, 850,
       851, 869, 876]
for val in index:
    jobs.loc[val,"job_salary"] = 113553.0

>**Data Analyst with Contract/Temp Role**

In [43]:
jobs[(jobs.job_title == 'data analyst') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index[0:150]

Int64Index([ 36,  40, 147, 148, 151, 152, 153, 161, 162, 164,
            ...
            746, 747, 783, 805, 806, 826, 846, 856, 860, 862],
           dtype='int64', length=105)

In [44]:
index=[36,40, 147, 148, 151, 152, 153, 161, 162, 164, 165, 167, 168, 172,
            176, 178, 179, 186, 187, 188, 191, 192, 193, 198, 201, 207, 208,
            210, 217, 218, 221, 222, 223, 224, 225, 226, 230, 231, 240, 242,
            243, 250, 251, 255, 256, 259, 260, 265, 269, 270,271, 272, 273, 275,
       277, 279, 281, 289, 291, 294, 296, 298, 303,
            312, 314, 315, 319, 321, 323, 324, 325, 327, 328, 329, 330, 337,
            338, 339, 340, 348, 349, 350, 351, 353, 354, 364, 367, 370, 518,
            556, 608, 632, 647, 695, 745, 746, 747, 783, 805, 806,826, 846, 856, 860, 862]
for val in index:
    jobs.loc[val,"job_salary"] = 123500.0

>**Data Engineer with Full Time Role**

In [45]:
jobs[(jobs.job_title == 'data engineer') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index[0:150]

Int64Index([ 29, 347, 366, 368, 371, 372, 373, 378, 379, 380,
            ...
            788, 804, 831, 852, 854, 857, 870, 872, 874, 878],
           dtype='int64', length=123)

In [48]:
index=[29, 347, 366, 368, 371, 372, 373, 378, 379, 380, 382, 386, 387,
            389, 390, 391, 392, 393, 394, 395, 396, 398, 400, 402, 403, 405,
            408, 420, 422, 424, 432, 435, 436, 437, 443, 448, 453, 457, 458,
            471, 476, 477, 479, 484, 485, 487, 488, 490, 491, 493,494, 495, 499, 503, 504, 505, 506, 508, 516, 520, 523, 526, 527,
            533, 534, 535, 536, 537, 538, 539, 541, 542, 550, 562, 563, 565,
            574, 575, 578, 579, 581, 582, 585, 595, 600, 601, 605, 606, 614,
            615, 618, 621, 622, 623, 624, 627, 629, 634, 635, 640,643, 645, 648, 649, 654, 661, 663, 664, 668, 672, 675, 676, 777,
            788, 804, 831, 852, 854, 857, 870, 872, 874, 878]
for val in index:
    jobs.loc[val,"job_salary"] = 221000.0

>**Data Engineer with Contract/Temp Role**

In [49]:
jobs[(jobs.job_title == 'data engineer') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index

Int64Index([ 27, 359, 381, 385, 401, 406, 421, 427, 428, 430, 434, 438, 441,
            444, 445, 447, 449, 450, 451, 459, 461, 465, 466, 468, 472, 474,
            481, 482, 483, 497, 507, 515, 559, 589, 609, 613, 619, 641, 644,
            646, 659, 766, 773, 801, 822, 823, 836, 837, 864, 875],
           dtype='int64')

In [50]:
index=[27, 359, 381, 385, 401, 406, 421, 427, 428, 430, 434, 438, 441,
            444, 445, 447, 449, 450, 451, 459, 461, 465, 466, 468, 472, 474,
            481, 482, 483, 497, 507, 515, 559, 589, 609, 613, 619, 641, 644,
            646, 659, 766, 773, 801, 822, 823, 836, 837, 864, 875]
for val in index:
    jobs.loc[val,"job_salary"] = 124999.5

>**Data Scientist with Full Time Role**

In [51]:
jobs[(jobs.job_title == 'data scientist') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index

Int64Index([  2,   3,   4,   6,   7,  11,  15,  19,  22,  28,  30,  32,  33,
             39,  41,  44,  45,  46,  47,  48,  51,  52,  54,  57,  59,  62,
             63,  67,  71,  73,  74,  87,  89,  93,  94,  97, 102, 104, 106,
            108, 109, 116, 117, 118, 120, 122, 124, 127, 131, 133, 138, 144,
            397, 399, 404, 413, 416, 417, 426, 431, 433, 452, 456, 463, 469,
            470, 496, 501, 502, 512, 513, 528, 543, 548, 555, 568, 576, 584,
            587, 596, 597, 599, 604, 607, 631, 636, 655, 656, 658, 660, 669,
            674, 865, 867, 877],
           dtype='int64')

In [52]:
index=[2,   3,   4,   6,   7,  11,  15,  19,  22,  28,  30,  32,  33,
             39,  41,  44,  45,  46,  47,  48,  51,  52,  54,  57,  59,  62,
             63,  67,  71,  73,  74,  87,  89,  93,  94,  97, 102, 104, 106,
            108, 109, 116, 117, 118, 120, 122, 124, 127, 131, 133, 138, 144,
            397, 399, 404, 413, 416, 417, 426, 431, 433, 452, 456, 463, 469,
            470, 496, 501, 502, 512, 513, 528, 543, 548, 555, 568, 576, 584,
            587, 596, 597, 599, 604, 607, 631, 636, 655, 656, 658, 660, 669,
            674, 865, 867, 877]
for val in index:
    jobs.loc[val,"job_salary"] = 140000.0

>**Data Scientist with Contract/Temp Role**

In [53]:
jobs[(jobs.job_title == 'data scientist') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index

Int64Index([ 43,  50,  55,  61,  65,  66,  69,  72,  76,  81,  82,  83,  84,
            101, 103, 105, 110, 113, 114, 115, 119, 135, 160, 446, 455, 522,
            572, 638, 827, 828],
           dtype='int64')

In [54]:
index=[43,  50,  55,  61,  65,  66,  69,  72,  76,  81,  82,  83,  84,
            101, 103, 105, 110, 113, 114, 115, 119, 135, 160, 446, 455, 522,
            572, 638, 827, 828]
for val in index:
    jobs.loc[val,"job_salary"] = 135000.0

>**Junior Data Analyst with Full Time Role**

In [74]:
jobs[(jobs.job_title == 'junior data analyst') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index

Int64Index([216, 241, 287], dtype='int64')

In [75]:
index=[216, 241, 287]

for val in index:
    jobs.loc[val,"job_salary"] = 52500.0

>**Junior Data Engineer with Full Time Role**

In [76]:
jobs[(jobs.job_title == 'junior data engineer') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index

Int64Index([423, 467, 486, 551, 752], dtype='int64')

In [77]:
index=[423, 467, 486, 551, 752]

for val in index:
    jobs.loc[val,"job_salary"] = 90000.0

>**Junior Data Engineer with Contract/Temp Role**

In [78]:
jobs[(jobs.job_title == 'junior data engineer') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index

Int64Index([842], dtype='int64')

In [79]:
index=[842]

for val in index:
    jobs.loc[val,"job_salary"] = 90000.0

>**Senior Business Intelligence with Full Time Role**

In [80]:
jobs[(jobs.job_title == 'senior business intelligence') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index

Int64Index([524, 560, 593, 701, 707, 708, 712, 727, 729, 736, 737, 751, 757,
            765, 793, 847, 859, 873],
           dtype='int64')

In [81]:
index=[524, 560, 593, 701, 707, 708, 712, 727, 729, 736, 737, 751, 757,
            765, 793, 847, 859, 873]

for val in index:
    jobs.loc[val,"job_salary"] = 135000.0

>**Senior Business Intelligence with Contract/Temp Role**

In [82]:
jobs[(jobs.job_title == 'senior business intelligence') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index

Int64Index([266, 713, 714, 724, 728, 784, 809, 861], dtype='int64')

In [83]:
index=[266, 713, 714, 724, 728, 784, 809, 861]

for val in index:
    jobs.loc[val,"job_salary"] = 135000.0

>**Senior Data Scientist with Full Time Role**

In [84]:
jobs[(jobs.job_title == 'senior data scientist') & (jobs.job_type == 'Full Time') & jobs.job_salary.isnull()].index

Int64Index([0, 9, 10, 24, 53, 56, 64, 78, 85, 95, 126, 129, 130, 132], dtype='int64')

In [85]:
index=[0, 9, 10, 24, 53, 56, 64, 78, 85, 95, 126, 129, 130, 132]

for val in index:
    jobs.loc[val,"job_salary"] = 150000.0

>**Senior Data Scientist with Contract/Temp Role**

In [86]:
jobs[(jobs.job_title == 'senior data scientist') & (jobs.job_type == 'Contract/Temp') & jobs.job_salary.isnull()].index

Int64Index([68], dtype='int64')

In [87]:
index=[68]

for val in index:
    jobs.loc[val,"job_salary"] = 260000.0

All done with salaries!!!!

In [88]:
jobs.job_salary.describe()

count       879.000000
mean     143387.554039
std       44223.403929
min       39000.000000
25%      113553.000000
50%      125000.000000
75%      150000.000000
max      370500.000000
Name: job_salary, dtype: float64

Having a look at the describe the min_salary : **\$39000** per year thats very less and max_salary goes upto : **\$710,000** far too high. I cross verified that I might have done some thing wrong but thats real.

**Categorizing the salaries in three bins low, medium and high**

In [89]:
jobs['salary_cat']=np.nan

In [90]:
jobs['salary_cat']=pd.cut(jobs.job_salary, [0,90000.00,125000.00,710000.00],labels=['low','medium','high'])

In [91]:
jobs.salary_cat.value_counts()

high      437
medium    395
low        47
Name: salary_cat, dtype: int64

**Dropping Duplicates if any**

In [92]:
jobs=jobs.drop_duplicates(keep= 'first')

In [93]:
jobs.head(2)

,job_area,job_classification,job_company,job_description,job_location,job_type,job_title,job_salary,salary_cat
0,CBD and Inner Suburbs,Science and Technology,SEEK Limited,The Opportunity We're looking for an experienc...,Melbourne,Full Time,senior data scientist,150000.0,high
1,Parramatta and Western Suburbs,Information and Communication Technology,ecareer employment services,Put Data to best useDesign & Develop BI Soluti...,Sydney,Full Time,senior data scientist,135000.0,high


In [94]:
#Handling outliers

In [95]:
q1= jobs.quantile(0.25)
q3= jobs.quantile(0.75)
IQR= q3 - q1

In [96]:
print(q1)
print(q3)
print(IQR)

job_salary    113553.0
Name: 0.25, dtype: float64
job_salary    150000.0
Name: 0.75, dtype: float64
job_salary    36447.0
dtype: float64


**Writing to a new CSV file for modelling**

In [77]:
jobs.to_csv("job_postings3.csv")